## Jupyter Metrics Final Project - Get Data

### Pull Down Data from GitHub

In [1]:
import pandas as pd
import datetime
from github import Github

In [ ]:
g = Github("jamieshq", "1ca9cfcf3a0f1922b95c582ff5fe5273d4c2a9a6")
cutoff = datetime.datetime(2015, 3, 30, 11, 38, 5, 291165)

In [ ]:
def get_prs(org, repo):
    prs = []
    print("Getting PRs for {}/{}".format(org.login, repo.name))
    for pr in repo.get_pulls(state="all"):
        if pr.created_at < cutoff:
            continue
        prs.append({
            'date': pr.created_at,
            'user': pr.user.login,
            'number': pr.number,
            'org': org.login,
            'repo': repo.name,
            'is_merged': pr.is_merged(),
            'state': pr.state
        })

    return prs

In [ ]:
def get_pr_comments(org, repo):
    comments = []
    print("Getting PR comments for {}/{}".format(org.login, repo.name))
    for pr in repo.get_pulls(state="all"):
        if pr.created_at < cutoff:
            continue
        for comment in pr.get_comments():
            comments.append({
                'date': comment.created_at,
                'user': comment.user.login,
                'number': pr.number,
                'org': org.login,
                'repo': repo.name
            })
    return comments

In [ ]:
def get_issues(org, repo):
    issues = []
    print("Getting issues for {}/{}".format(org.login, repo.name))
    for issue in repo.get_issues(state="all"):
        if issue.created_at < cutoff:
            continue
        issues.append({
            'date': issue.created_at,
            'user': issue.user.login,
            'number': issue.number,
            'org': org.login,
            'repo': repo.name,
            'state': issue.state
        })
            
    return issues

In [ ]:
def get_issue_comments(org, repo):
    comments = []
    print("Getting issue comments for {}/{}".format(org.login, repo.name))
    for issue in repo.get_issues(state="all"):
        if issue.created_at < cutoff:
            continue
        for comment in issue.get_comments():
            
            comments.append({
                'issue_date': issue.created_at,
                'user': issue.user.login,
                'number': issue.number,
                'org': org.login,
                'repo': repo.name,
                'comments': comment.body,
                'comment_creation_date' : comment.created_at
                    
            })
        
    return comments

In [ ]:
#prs = []
#pr_comments = []
issues = []
issue_comments = []

test_orgs = ["jupyter-resources"]
real_orgs = ["jupyterlab", "jupyterhub", "jupyter-widgets", "jupyter-incubator"]
for org_name in real_orgs:
    org = g.get_organization(org_name)
    for repo in org.get_repos():
        #prs.extend(get_prs(org, repo))
        #pr_comments.extend(get_pr_comments(org, repo))
        #issues.extend(get_issues(org, repo))
        issue_comments.extend(get_issue_comments(org, repo))

In [ ]:
issue_comments = pd.DataFrame(issue_comments).set_index(['org', 'repo', 'number', 'issue_date','comment_creation_date', 'comments']).sortlevel()

In [ ]:
issue_comments.to_csv("issue_comments.csv", encoding="utf-8")

Converting an epoch timestamp to datetime after querying for ratelimit reset time
I hit my GitHub ratelimit after reading through the first organization listed (Jupyter). I used the below curl command to learn my ratelimit reset time, which was about an hour after receiving the timeout.
curl -i https://api.github.com/users/username
HTTP/1.1 200 OK
Date: Mon, 01 Jul 2013 17:27:06 GMT
Status: 200 OK
X-RateLimit-Limit: 60
X-RateLimit-Remaining: 56
X-RateLimit-Reset: 1372700873
Sources: https://developer.github.com/v3/#rate-limiting http://stackoverflow.com/questions/12400256/python-converting-epoch-time-into-the-datetime

In [2]:
import datetime
datetime.datetime.fromtimestamp(1491196314).strftime('%c')

'Sun Apr  2 22:11:54 2017'